[View in Colaboratory](https://colab.research.google.com/github/brucecmd/learn_gluon/blob/master/AlexNet.ipynb)

In [0]:
from mxnet.gluon import nn
import mxnet as mx
from mxnet.gluon import loss as gloss
from time import time
from mxnet.gluon import data as gdata
from mxnet import gluon, init, autograd, nd

In [0]:

###LeNet
#net = nn.Sequential()
#net.add(
#nn.Conv2D(channels=6, kernel_size=5, activation='sigmoid'),
#nn.MaxPool2D(pool_size=2, strides=2),
#nn.Conv2D(channels=16, kernel_size=5, activation='sigmoid'),
#nn.MaxPool2D(pool_size=2, strides=2),
#nn.Dense(120, activation='sigmoid'),
#nn.Dense(84, activation='sigmoid'),
#nn.Dense(10)
#)

# AlexNet
net = nn.Sequential()
net.add(nn.Conv2D(96, kernel_size=11, strides=4, activation='relu'))
net.add(nn.MaxPool2D(pool_size=3,strides=2))

net.add(nn.Conv2D(channels=256, kernel_size=5, padding=2, activation='relu'))
net.add(nn.MaxPool2D(pool_size=3, strides=2))

net.add(nn.Conv2D(channels=384, kernel_size=3, padding=1, activation='relu'))
net.add(nn.Conv2D(channels=384, kernel_size=3, padding=1, activation='relu'))
net.add(nn.Conv2D(channels=256, kernel_size=3, padding=1, activation='relu'))
net.add(nn.MaxPool2D(pool_size=3, strides=2))

#net.add(nn.Dense(4096, activation='relu'))
#net.add(nn.Dropout(0.5))

#net.add(nn.Dense(4096, activation='relu'))
#net.add(nn.Dropout(0.5))

net.add(nn.Dense(10))

net.initialize(init.Xavier()) # 要注意的是，这里的sigma选太小的话，训不出结果来nn
#print(net)

In [7]:
X = nd.random.uniform(shape=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv5 output shape:	 (1, 96, 54, 54)
pool3 output shape:	 (1, 96, 26, 26)
conv6 output shape:	 (1, 256, 26, 26)
pool4 output shape:	 (1, 256, 12, 12)
conv7 output shape:	 (1, 384, 12, 12)
conv8 output shape:	 (1, 384, 12, 12)
conv9 output shape:	 (1, 256, 12, 12)
pool5 output shape:	 (1, 256, 5, 5)
dense1 output shape:	 (1, 10)


In [0]:
mnist_train = gdata.vision.FashionMNIST(train=True)
mnist_test = gdata.vision.FashionMNIST(train=False)

In [0]:
batch_size = 256
transformer = []
transformer += [gdata.vision.transforms.Resize(224)] # 要先resize再totensor
transformer += [gdata.vision.transforms.ToTensor()]
transformer = gdata.vision.transforms.Compose(transformer)
train_iter = gdata.DataLoader(mnist_train.transform_first(transformer), batch_size, shuffle=True)
test_iter = gdata.DataLoader(mnist_test.transform_first(transformer), batch_size, shuffle=True)

In [0]:
def accuracy(y_hat, y):
    return (y_hat.argmax(axis=1)==y.astype('float32')).mean().asscalar()

  
def evaluate_accuracy(data_iter, net):
    acc = nd.array([0])
    for x, y in data_iter:
        acc += accuracy(net(x), y)
    return acc.asscalar() / len(data_iter)

In [0]:
loss_func = gloss.SoftmaxCrossEntropyLoss()

In [0]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate':0.1})

In [0]:
epochs = 10
batch_size = 256

for i in range(epochs):
    for data, label in train_iter:
        with autograd.record():
            y_hat = net(data)
            l = loss_func(y_hat, label)
        l.backward()
        trainer.step(batch_size)
    train_acc = evaluate_accuracy(train_iter, net)
    test_acc = evaluate_accuracy(test_iter, net)
    print('epoch[%d], train acc[%f], test acc[%f]'%(i, train_acc, test_acc))